In [1]:
import sys
sys.path.append("../")
import io
from glob import glob
from tqdm.contrib.concurrent import process_map
from document_search.ocr import DocumentReader, EntityProcessor


/opt/conda/envs/doc-search/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Adding all documents to storage

In [2]:
documents = glob('../data/*.pdf')
len(documents)


17

In [3]:
%%time

def read_document(document_path):
    doc_reader = DocumentReader()
    with open(document_path, 'rb') as f:
        raw_document = io.BytesIO(f.read())
    processed_doc, errors = doc_reader.read(document_path)
    print(document_path, errors, len(processed_doc.entities))
    processed_doc.entities = EntityProcessor.merge_text_entities(processed_doc.entities)
    processed_doc.entities = EntityProcessor.filter_short_entities(processed_doc.entities)
    return processed_doc, raw_document


doc_reader = DocumentReader()

raw_documents = []
processed_docs = []
for processed_doc, raw_document in map(read_document, documents): 
    raw_documents.append(raw_document)
    processed_docs.append(processed_doc)


/opt/conda/envs/doc-search/lib/python3.10/site-packages/easyocr/detection.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(copyStateDict(torch.load(t

../data/Postanovlenie-Pravitelstva-RF-_-87_red-ot-27.05.2022_.pdf [] 2501
../data/SP-51.13330.2011.pdf [] 1901
../data/SP-60.13330.2020.pdf [] 3325
../data/SP.13130.2023.pdf [] 1873
../data/ГОСТ 21.101-2020.pdf [] 3461
../data/ГОСТ 27751-2014.pdf [] 1356
../data/ГОСТ 5781-82.pdf [] 925
../data/Постановление_Правительства_РФ_от_16_09_2020_N_1479_Об_утверждении.pdf [] 1881
../data/СНиП 12-01-2004 (СП 48.13330.2019).pdf [] 3430
../data/СП 20.13330.2016.pdf [] 6147
../data/СП 50.13330.2012.pdf [] 8485
../data/СП 52.13330.2016.pdf [] 1114
../data/СП50.13330.2012 Тепловая защита зданий.pdf [] 108389
../data/СП_22_13330_2016_«СНиП_2_02_01_83_Основания_зданий_и_сооружений».pdf [] 1646
../data/СП_54_13330_2016_«СНиП_31_01_2003_Здания_жилые_многоквартирные».pdf [] 950
../data/СанПиН 2.1.2.2645-10.pdf [] 897
../data/СанПиН 2.2.1.pdf [] 206
CPU times: user 3h 28min 57s, sys: 35.3 s, total: 3h 29min 32s
Wall time: 16min 57s


In [4]:
from document_search.storages import DocumentStorageE5
from document_search.search import TextEntityEmbedderE5

embedder = TextEntityEmbedderE5()
storage = DocumentStorageE5(embedder)


/home/jovyan/nfs/igor/hacks/document-search/notebooks/../document_search/search/embedders.py:11: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  self.embedder = HuggingFaceEmbeddings(


In [5]:
for doc, raw_doc in zip(processed_docs, raw_documents):
    storage.add_document(doc, raw_doc, pbar=True)


91it [00:01, 50.14it/s]                        
55it [00:01, 49.09it/s]                        
121it [00:01, 67.89it/s]                         
63it [00:01, 52.98it/s]                        
130it [00:03, 40.94it/s]                         
58it [00:01, 29.44it/s]                        
100%|██████████| 32/32 [00:00<00:00, 41.20it/s]
90it [00:01, 60.99it/s]                        
137it [00:03, 39.24it/s]                         
175it [00:03, 47.84it/s]                         
201it [00:04, 45.10it/s]                         
38it [00:01, 24.08it/s]                        
195it [00:05, 38.84it/s]                         
55it [00:02, 21.77it/s]                        
100%|██████████| 26/26 [00:00<00:00, 30.13it/s]
32it [00:01, 30.65it/s]                        
9it [00:00, 38.12it/s]                       


## Test search

In [6]:
from document_search.rag.retrievers import YandexGPTRetriever

rag_retriever = YandexGPTRetriever(embedder=embedder, storage=storage)


/home/jovyan/nfs/igor/hacks/document-search/notebooks/../document_search/rag/retrievers.py:26: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  self.llm_chain = LLMChain(llm=self.llm, prompt=self.prompt)


In [7]:
query = "какие трубопроводы не допускается прокладывать через помещения вентиляционного оборудования?"

data, answer = rag_retriever.retrieve_answer_detailed(query, k=5, rag_k=1, context_length=0)
print(answer)


/home/jovyan/nfs/igor/hacks/document-search/notebooks/../document_search/rag/retrievers.py:67: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  answer = self.llm_chain.run(


**Ответ:**

Через помещение для вентиляционного оборудования не допускается прокладывать трубопроводы:
а) с легковоспламеняющимися и горючими жидкостями и газами;
б) канализационные с прочистками и ревизиями (кроме трубопроводов ливневой канализации и водоотведения из вышележащих помещений для вентиляционного оборудования, в том числе от вентиляционного оборудования); допускается прокладка канализационных трубопроводов на хомутовых безраструбных соединениях.

В предоставленном контексте нет информации о том, какие ещё трубопроводы нельзя прокладывать через помещения вентиляционного оборудования.

Таким образом, исходя из имеющейся информации, можно сделать вывод, что трубопроводы с легковоспламеняющимися и горючими жидкостями, а также канализационные трубопроводы с прочистками и ревизиями запрещено прокладывать через помещение для вентиляционного оборудования. Однако для точного ответа на вопрос пользователя необходимо обратиться к более широкому кругу источников или уточнить запрос.


In [8]:
data


[(TextDocEntity(position=EntityPosition(document_name='SP-60.13330.2020.pdf', page_number=66, document_id='63d55c76804640a98d1ebbcf281b68a5'), text='10.6  Через  помещение  для  вентиляционного  оборудования  не  допускается  прокладывать трубопроводы:'),
  0.91254383),
 (TextDocEntity(position=EntityPosition(document_name='SP-60.13330.2020.pdf', page_number=77, document_id='63d55c76804640a98d1ebbcf281b68a5'), text='Не  допускается  прокладка  магистральных  и  разводящих  трубопроводов  систем  отопления  и внутреннего теплоснабжения через помещения жилых квартир, а также установка в них арматуры и спускных устройств общедомовых систем.'),
  0.8962804),
 (TextDocEntity(position=EntityPosition(document_name='SP-60.13330.2020.pdf', page_number=50, document_id='63d55c76804640a98d1ebbcf281b68a5'), text='7.10.14  Оборудование  вытяжных  систем  производственных,  административно-бытовых  и общественных  зданий,  удаляющих  воздух  с  резким  или  неприятным  запахом  (из  общественных убор

## Run tests

In [9]:
import pandas as pd

test_data = pd.read_csv('test_data.csv')
for test_case in test_data.to_dict('records'):
    query = test_case['question']
    data, answer = rag_retriever.retrieve_answer_detailed(query, k=5, rag_k=1, context_length=0)
    print('-'*40)
    print('QUERY:')
    print(query)
    print('ANSWER:')
    print(answer)
    print('SEARCH DATA:')
    for res in data:
        print(res)
    print("GROUND TRUTH:")
    print(test_case['ground_truth'])


----------------------------------------
QUERY:
Необходимо ли отражать перечень помещений и оборудования, подлежащих защите автоматическими установками пожаротушения при разработке проектной документации объекта капитального строительства согласно 87 постановлению?
ANSWER:
**Ответ:**

Перечень помещений и оборудования, подлежащих защите автоматическими установками пожаротушения, определяется на основе расчёта пожарных рисков и установленных нормативными документами требований пожарной безопасности.

Согласно постановлению № 963 от 27.05.2022, в текстовой части раздела 9 «Мероприятия по обеспечению пожарной безопасности» проектной документации должна быть представлена информация о категории зданий, сооружений, помещений, оборудования и наружных установок по признаку взрывопожарной и пожарной опасности. В том числе должен быть приведён перечень зданий, сооружений, помещений и оборудования, которые подлежат защите автоматическими установками пожаротушения и оборудованию автоматической пож